In [1]:
import numpy as np
import sympy as sp

In [7]:
np.random.normal(loc=0.0, scale=1.0, size=None) #loc:평균, scale:분산

0.09015030908781119

In [8]:
np.random.uniform(low=0.0, high=1.0, size=None)

0.9094360603045485

In [21]:
# Rosenbrock
def f(x, a=1, b=5):
    y = (a - x[0])**2 + b*(x[1] - x[0]**2)**2
    return y

In [22]:
x = sp.IndexedBase('x')
gradients = np.array([sp.diff(f(x), x[i]) for i in range(2)])
grads = sp.lambdify(x, gradients, 'numpy')

x_ = [-2., 2.]

In [48]:
def NoisyDescent(f, grads, x, alpha, maxiter=1000, TOL=1e-4, verbose=True):
    
    y_prev = f(x)
    
    flag = True
    i = 1
    
    while flag:
        g = np.asarray(grads(x))
        g = -g / np.sqrt(np.dot(g, g))
        
        noise = np.random.normal(loc=0, scale=1./i, size=g.shape)
        
        
        x = x + alpha*g + noise
        y = f(x)
        
        if verbose:
            print(f'{i}: y {y:.4f}, x {x}')
            
        if abs(y_prev - y) < TOL * (abs(y_prev) + TOL) or i >= maxiter:
            
            flag = False
            
        y_prev = y
        
        i += 1
        
    return i, y, x

In [49]:
NoisyDescent(f, grads, x_, alpha=0.01)

1: y 514.7600, x [-3.26266247  0.67912797]
2: y 1037.4240, x [-3.86901367  0.7304699 ]
3: y 1402.2819, x [-4.11517589  0.34481472]
4: y 1042.8128, x [-3.81379731  0.26471251]
5: y 1100.3552, x [-3.88539888  0.42330103]
6: y 1052.4108, x [-3.82864243  0.31210975]
7: y 916.2692, x [-3.69228655  0.25948621]
8: y 1066.5533, x [-3.82747691  0.20486549]
9: y 1127.3422, x [-3.87360449  0.14822968]
10: y 939.6855, x [-3.6963012   0.11546382]
11: y 912.3796, x [-3.66616271  0.09453966]
12: y 898.2665, x [-3.66601557  0.199619  ]
13: y 875.2415, x [-3.63439126  0.14155925]
14: y 783.5639, x [-3.53679184  0.15590978]
15: y 861.0950, x [-3.61192631  0.08587909]
16: y 879.2109, x [-3.63941384  0.14810665]
17: y 831.2271, x [-3.5889971   0.15165658]
18: y 798.4526, x [-3.55056994  0.13461903]
19: y 833.7278, x [-3.58236391  0.08398013]
20: y 807.8925, x [-3.55704028  0.10561628]
21: y 794.6393, x [-3.54419065  0.11950593]
22: y 749.2763, x [-3.49183402  0.11731199]
23: y 679.3594, x [-3.39907583  0.

663: y 0.0008, x [0.99056059 0.99327659]
664: y 0.0008, x [1.0009287  0.98954856]
665: y 0.0007, x [0.99149498 0.99406288]
666: y 0.0011, x [1.00253899 0.99028073]
667: y 0.0000, x [0.99679601 0.99287594]
668: y 0.0032, x [0.98641066 0.9977065 ]
669: y 0.0000, x [0.99748981 0.99437214]
670: y 0.0027, x [0.98921487 1.00118883]
671: y 0.0001, x [0.9966463  0.99686078]
672: y 0.0027, x [1.00762129 0.99251813]
673: y 0.0000, x [0.99874077 1.00021773]
674: y 0.0027, x [1.0079461  0.99293672]
675: y 0.0000, x [0.99831783 0.99745776]
676: y 0.0012, x [1.00404115 0.99259506]
677: y 0.0001, x [0.99691101 0.99782924]
678: y 0.0008, x [1.00297649 0.9930144 ]
679: y 0.0002, x [0.99489635 0.99649819]
680: y 0.0018, x [1.00566536 0.99257008]
681: y 0.0001, x [0.99779576 0.99955277]
682: y 0.0014, x [1.00674941 0.99711173]
683: y 0.0000, x [0.9982218  0.99810064]
684: y 0.0018, x [1.00642809 0.9943112 ]
685: y 0.0008, x [0.99440007 1.00151161]
686: y 0.0005, x [1.00313377 0.99667601]
687: y 0.0005, x

(1001, 0.0008062880937549988, array([0.9947648 , 1.00203803]))

In [42]:
y_best = 1e+8
x_best = [0., 0.]

In [43]:
for i in range(10):
    i, y, x = NoisyDescent(f, grads, x_, alpha=0.01, verbose=False)

    if y < y_best:
        y_best, x_best = y, x

print(y_best, x_best)

0.00012242652382270312 [1.00398804 1.00337632]


In [88]:
def SimulatedAnnealing(f, x, t, schedule, decay=0.9, maxiter=1000, TOL=1e-4):

    y = f(x)
    y_best, x_best = y, x
    t0 = t
    
    flag = True
    i = 1
    
    while flag:
        
        x_ = x + np.random.normal(size=len(x))
        y_ = f(x_)
        dy = y_ - y
        
        # clip
        t = max(t, 0.1)
        
        if dy < 0:
            dy = max(-10, dy)
        else:
            dy = min(10, dy)
            
        if y_ <= 0 or np.random.uniform() < min(np.exp(-dy/t), 1):
            x, y = x_, y_
            
        if y_ < y_best:
            y_best, x_best = y_, x_
            
        if schedule == 'logarithmic':
            t = t * np.log(2) / np.log(i+1)
        elif schedule == 'exponential':
            t = decay * t
        elif schedule == 'fast':
            t = t0 / i 
            
        if i >= maxiter:
            flag = False
            
        i += 1
        
    return i, y_best, x_best

In [99]:
t = 1
SimulatedAnnealing(f, x_, t, 'logarithmic')

(1001, 0.0024739220502673605, array([1.0489559 , 1.09637805]))

In [105]:
SimulatedAnnealing(f, x_, t, 'exponential')

(1001, 0.0019102418552597407, array([1.04327116, 1.09116603]))

In [113]:
SimulatedAnnealing(f, x_, t, 'fast')

(1001, 0.005327011292896992, array([0.99349147, 1.01953577]))